# Weights and Biases Logger

This notebook logs the evaluation metrics of dev set and test set on every model.

In [1]:
import pandas as pd
import wandb
from configparser import ConfigParser

c:\Users\Cheyanne\miniconda3\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Cheyanne\miniconda3\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Cheyanne\miniconda3\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


.cfg file is a config file that store your personal api key
The format inside the file looks like this:
```
[<YOUR_API_KEY_NAME>]
auth_key: <HERE_IS_YOUR_API_KEY>
```
.gitignore is set on .cfg

In [2]:
parser = ConfigParser()
_ = parser.read("../../../notebook.cfg")
wandb_api_auth_key = parser.get("wandb_api_key", "auth_key")

In [3]:
wandb.login(key=wandb_api_auth_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cheyannelam. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Cheyanne\.netrc


True

# Validation set

In [4]:
# Initialize WandB
def wandb_log_eval_metrics(file, data_src, sample_portion, name, tags, job_type='eval', project='BioLaySumm2024', entity='bossy_beaver'):
    run_config = {'data_src':data_src, 'sample_portion':sample_portion}
    run = wandb.init(project=PROJECT_NAME, entity=ENTITY, 
                     job_type=job_type, tags=tags, name=name)
    
    # Open the file in read mode
    with open(PATH_TO_VAL_METRICS + "/" + file, 'r') as file:
        
        lines = file.readlines()
    
    # Create an empty dictionary to store the metrics
    metrics = {}
    
    # Iterate over each line
    for line in lines:
        # Split the line into key and value using ':' as the delimiter
        key, value = line.strip().split(': ')
        # Store the key-value pair in the dictionary
        metrics[key] = float(value)
    print(metrics)
    
    # Log metrics to WandB
    run.log(metrics)
    
    # Finish WandB run
    run.finish()

In [8]:
model_name = 'biomistral' #'mistral7b_awq'
data_type = 'mini_dev' #'test' #
method = '3shots' #3, 1
withtag = 'notag' #withtag' # 

PROJECT_NAME = 'BioLaySumm2024'# wandb project name
ENTITY = 'bossy_beaver' #change to your wandb team name
if withtag == 'notag':
    tag_name = ''
else:
    tag_name = '_'+withtag

if data_type == 'mini_dev':
    data_name = 'val_mini'
else:
    data_name = data_type
PATH_TO_VAL_METRICS = f'./{model_name}/{method}_with200to300words{tag_name}/{data_name}'
VAL_METRICS_FILES = ['elife_scores.txt', 'plos_scores.txt', 'scores.txt']

SAMPLE_PORTION = 0.1 if data_type == 'mini_dev' else 1

In [9]:
# For example we can use these parameters to define our metrics
# file = 'elife_scores.txt'
# data_src = 'eLife_dev'
# sample_portion = 0.1
# name = 'dummy_baseline'
# tags = ['baseline']

In [10]:
# eLife dummy baseline on 10% dev set
wandb_log_eval_metrics(file='elife_scores.txt', data_src=f"eLife_{data_type.split('_')[-1]}", sample_portion=SAMPLE_PORTION, 
#                       name=f'elife_{model_name}_{method}_{withtag}_{data_type}', tags=[model_name.replace('_', ''), data_type.replace('_', ''), method.replace('_', ''), withtag, 'elife'])
# PLOS dummy baseline on 10% dev set
wandb_log_eval_metrics(file='plos_scores.txt', data_src=f"PLOS_{data_type.split('_')[-1]}", sample_portion=SAMPLE_PORTION, 
                       name=f'plos_{model_name}_{method}_{withtag}_{data_type}', tags=[model_name.replace('_', ''), data_type.replace('_', ''), method.replace('_', ''), withtag, 'plos'])
# Combined dummy baseline on 10% dev set
wandb_log_eval_metrics(file='scores.txt', data_src=f"combined_{data_type.split('_')[-1]}", sample_portion=SAMPLE_PORTION, 
                       name=f'combined_{model_name}_{method}_{withtag}_{data_type}', tags=[model_name.replace('_', ''), data_type.replace('_', ''), method.replace('_', ''), withtag, 'combined'])

{'ROUGE1': 0.32027959070350637, 'ROUGE2': 0.09184691865968388, 'ROUGEL': 0.29541749140308293, 'BERTScore': 0.8321343038393103, 'FKGL': 14.853623188405798, 'DCRS': 9.642246376811595, 'CLI': 14.535289855072465, 'LENS': 43.83759635231857, 'AlignScore': 0.6364608373276998, 'SummaC': 0.5903487106164297}


AlignScore,▁
BERTScore,▁
CLI,▁
DCRS,▁
FKGL,▁
LENS,▁
ROUGE1,▁
ROUGE2,▁
ROUGEL,▁
SummaC,▁
AlignScore,0.63646


{'ROUGE1': 0.30190527232905584, 'ROUGE2': 0.06967799728775645, 'ROUGEL': 0.28020369463424355, 'BERTScore': 0.826048294649176, 'FKGL': 14.4768115942029, 'DCRS': 9.779873188405798, 'CLI': 14.783686594202898, 'LENS': 49.78068605668234, 'AlignScore': 0.6334309402745271, 'SummaC': 0.5879519823938608}


AlignScore,▁
BERTScore,▁
CLI,▁
DCRS,▁
FKGL,▁
LENS,▁
ROUGE1,▁
ROUGE2,▁
ROUGEL,▁
SummaC,▁
AlignScore,0.63343


# Test Set

In [25]:
PATH_TO_VAL_METRICS = './scoring_result'
TEST_METRICS_FILES = ['elife_scores.txt', 'plos_scores.txt', 'scores.txt']

In [27]:
# elife dummy baseline on test set
wandb_log_eval_metrics(file='elife_scores.txt', data_src='eLife_test', sample_portion=1, 
                       name='elife_dummy_baseline_test', tags=['baseline', 'test'])

{'ROUGE1': 0.3089138993381871, 'ROUGE2': 0.06957613212293816, 'ROUGEL': 0.2813751343330106, 'BERTScore': 0.8407322109585077, 'FKGL': 15.080985915492958, 'DCRS': 11.753802816901409, 'CLI': 17.258943661971827, 'LENS': 30.24145106148018, 'AlignScore': 0.9781640558175637, 'SummaC': 0.9402011343291108}


AlignScore,▁
BERTScore,▁
CLI,▁
DCRS,▁
FKGL,▁
LENS,▁
ROUGE1,▁
ROUGE2,▁
ROUGEL,▁
SummaC,▁
AlignScore,0.97816


In [28]:
# PLOS dummy baseline on test set
wandb_log_eval_metrics(file='plos_scores.txt', data_src='PLOS_test', sample_portion=1, 
                       name='plos_dummy_baseline_test', tags=['baseline', 'test'])

{'ROUGE1': 0.461164102179743, 'ROUGE2': 0.16057957580294918, 'ROUGEL': 0.4178593622680423, 'BERTScore': 0.8693121696861696, 'FKGL': 14.751408450704227, 'DCRS': 11.793098591549297, 'CLI': 16.704154929577463, 'LENS': 35.30267832931986, 'AlignScore': 0.9772088964220503, 'SummaC': 0.9488216630170043}


AlignScore,▁
BERTScore,▁
CLI,▁
DCRS,▁
FKGL,▁
LENS,▁
ROUGE1,▁
ROUGE2,▁
ROUGEL,▁
SummaC,▁
AlignScore,0.97721


In [29]:
# Combined dummy baseline on test set
wandb_log_eval_metrics(file='scores.txt', data_src='combined_test', sample_portion=1, 
                       name='dummy_baseline_test', tags=['baseline', 'combined', 'test'])

{'ROUGE1': 0.385039000758965, 'ROUGE2': 0.11507785396294368, 'ROUGEL': 0.34961724830052643, 'BERTScore': 0.8550221903223387, 'FKGL': 14.916197183098593, 'DCRS': 11.773450704225354, 'CLI': 16.981549295774645, 'LENS': 32.77206469540002, 'AlignScore': 0.977686476119807, 'SummaC': 0.9445113986730576}


AlignScore,▁
BERTScore,▁
CLI,▁
DCRS,▁
FKGL,▁
LENS,▁
ROUGE1,▁
ROUGE2,▁
ROUGEL,▁
SummaC,▁
AlignScore,0.97769
